# Data Aggregation and Group Operations

In [1]:
import numpy as np
from numpy.random import randn

import pandas as pd
from pandas import Series, DataFrame

## GroupBy Mechanics

In [2]:
df = DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['A','B','A','B','A'],
    'data1': np.arange(0,5),
    'data2': np.arange(0,10,2)
})
df

,data1,data2,key1,key2
0,0,0,a,A
1,1,2,a,B
2,2,4,b,A
3,3,6,b,B
4,4,8,a,A


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     A       2
      B       1
b     A       2
      B       3
Name: data1, dtype: int64

In [6]:
means.unstack()

key2,A,B
key1,,
a,2,1
b,2,3


In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.666667,3.333333
b,2.500000,5.000000


In [8]:
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    A         2      4
     B         1      2
b    A         2      4
     B         3      6

### Iterating Over Groups

In [9]:
for name, group in df.groupby('key1'):
    print name
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [10]:
for type, group in df.groupby(df.dtypes, axis=1):
    print type
    print group

int64
   data1  data2
0      0      0
1      1      2
2      2      4
3      3      6
4      4      8
object
  key1 key2
0    a    A
1    a    B
2    b    A
3    b    B
4    a    A


### Selecting a Column or Subset of Columns

In [11]:
df.groupby('key1')['data1'].count()

key1
a    3
b    2
Name: data1, dtype: int64

In [12]:
df['data1'].groupby(df['key1']).count()

key1
a    3
b    2
Name: data1, dtype: int64

### Grouping with Dicts and Series

In [13]:
people = DataFrame(
    np.arange(25).reshape((5,5)),
    columns=['a','b','c','d','e'],
    index=['one','two','three','four','five']
)
people

,a,b,c,d,e
one,0,1,2,3,4
two,5,6,7,8,9
three,10,11,12,13,14
four,15,16,17,18,19
five,20,21,22,23,24


In [14]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

people.groupby(mapping, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


In [15]:
map_series = Series(mapping)
people.groupby(map_series, axis=1).sum()

,blue,red
one,5,5
two,15,20
three,25,35
four,35,50
five,45,65


### Grouping with Functions

In [16]:
people.groupby(len).sum()

,a,b,c,d,e
3,5,7,9,11,13
4,35,37,39,41,43
5,10,11,12,13,14


In [17]:
people.groupby(lambda x: x[0]).sum()

,a,b,c,d,e
f,35,37,39,41,43
o,0,1,2,3,4
t,15,17,19,21,23


### Grouping by Index Levels

In [18]:
columns = pd.MultiIndex.from_arrays(
    [['US','US','US','JP','JP'],
    [1,3,5,1,3]], names=['city', 'tenor'])

hier_df = DataFrame(np.arange(25).reshape((5,5)),
                   columns=columns)
hier_df

city   US          JP    
tenor   1   3   5   1   3
0       0   1   2   3   4
1       5   6   7   8   9
2      10  11  12  13  14
3      15  16  17  18  19
4      20  21  22  23  24

In [19]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
4,2,3


## Data Aggregation

In [20]:
grouped = df.groupby('key1')
for key, group in grouped:
    print key
    print group

a
   data1  data2 key1 key2
0      0      0    a    A
1      1      2    a    B
4      4      8    a    A
b
   data1  data2 key1 key2
2      2      4    b    A
3      3      6    b    B


In [21]:
grouped['data1'].quantile(0.5)

key1
a    1.0
b    2.5
Name: data1, dtype: float64

In [22]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,4,8
b,1,2


In [23]:
grouped['data1'].mean()

key1
a    1.666667
b    2.500000
Name: data1, dtype: float64

In [24]:
grouped['data1'].median()

key1
a    1.0
b    2.5
Name: data1, dtype: float64

### Column-wise and Multiple Function Application

In [25]:
tips = pd.read_csv('ch08/tips.csv')
tips[:3]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [26]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:3]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587


In [27]:
grouped = tips.groupby(['sex','smoker'])

In [28]:
grouped['tip_pct'].agg('mean').unstack()

smoker,No,Yes
sex,,
Female,0.156921,0.182150
Male,0.160669,0.152771


In [29]:
grouped['tip_pct'].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [30]:
grouped['tip_pct'].agg([
    ('avarage', 'mean'), ('standard deviation','std')
])

avarage  standard deviation
sex    smoker                              
Female No      0.156921            0.036421
       Yes     0.182150            0.071595
Male   No      0.160669            0.041849
       Yes     0.152771            0.090588

In [31]:
grouped['tip_pct', 'total_bill'].agg(['mean', 'std'])

tip_pct           total_bill          
                   mean       std       mean       std
sex    smoker                                         
Female No      0.156921  0.036421  18.105185  7.286455
       Yes     0.182150  0.071595  17.977879  9.189751
Male   No      0.160669  0.041849  19.791237  8.726566
       Yes     0.152771  0.090588  22.284500  9.911845

In [32]:
grouped.agg({'tip': 'max', 'size': 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

### Returning Aggregated Data in “unindexed” Form

In [33]:
tips.groupby(['sex','smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


## Group-wise Operations and Transformations

In [34]:
df

,data1,data2,key1,key2
0,0,0,a,A
1,1,2,a,B
2,2,4,b,A
3,3,6,b,B
4,4,8,a,A


In [35]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,1.666667,3.333333
b,2.500000,5.000000


In [36]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0,0,a,A,1.666667,3.333333
1,1,2,a,B,1.666667,3.333333
4,4,8,a,A,1.666667,3.333333
2,2,4,b,A,2.500000,5.000000
3,3,6,b,B,2.500000,5.000000


In [37]:
people.groupby(len).mean()

,a,b,c,d,e
3,2.5,3.5,4.5,5.5,6.5
4,17.5,18.5,19.5,20.5,21.5
5,10.0,11.0,12.0,13.0,14.0


In [38]:
people.groupby(len).transform('mean')

,a,b,c,d,e
one,2.5,3.5,4.5,5.5,6.5
two,2.5,3.5,4.5,5.5,6.5
three,10.0,11.0,12.0,13.0,14.0
four,17.5,18.5,19.5,20.5,21.5
five,17.5,18.5,19.5,20.5,21.5


In [39]:
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(len).transform(demean)
demeaned

,a,b,c,d,e
one,-2.5,-2.5,-2.5,-2.5,-2.5
two,2.5,2.5,2.5,2.5,2.5
three,0.0,0.0,0.0,0.0,0.0
four,-2.5,-2.5,-2.5,-2.5,-2.5
five,2.5,2.5,2.5,2.5,2.5


### Apply: General split-apply-combine

In [40]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [41]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [42]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [43]:
frame = DataFrame({'data1': randn(1000), 'data2': randn(1000)})

In [44]:
factor = pd.cut(frame.data1, 4)
factor[:4]

0     (-0.0237, 1.524]
1    (-1.571, -0.0237]
2    (-1.571, -0.0237]
3    (-1.571, -0.0237]
Name: data1, dtype: category
Categories (4, object): [(-3.125, -1.571] < (-1.571, -0.0237] < (-0.0237, 1.524] < (1.524, 3.071]]

In [45]:
grouped = frame['data2'].groupby(factor)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
"(-3.125, -1.571]",54.0,0.138576,0.999545,-2.339381,-0.598910,-0.001090,0.728273,2.606072
"(-1.571, -0.0237]",441.0,0.050533,1.046139,-3.655366,-0.643253,0.073107,0.745309,3.004693
"(-0.0237, 1.524]",453.0,-0.006194,0.998232,-2.994061,-0.645441,-0.019315,0.660702,2.665165
"(1.524, 3.071]",52.0,0.048714,0.869034,-2.281366,-0.680666,0.127108,0.839789,1.628722


In [46]:
grouping = pd.qcut(frame.data1, 4, labels=False)
grouping[:4]

0    2
1    0
2    0
3    0
Name: data1, dtype: int64

In [47]:
grouped = frame['data2'].groupby(grouping)
grouped.apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
0,250.0,0.124342,1.030236,-3.655366,-0.560786,0.133642,0.779550,3.004693
1,250.0,0.003605,1.048431,-2.949639,-0.696761,0.005921,0.674192,2.651193
2,250.0,0.000568,1.036169,-2.994061,-0.607579,-0.009180,0.700600,2.665165
3,250.0,-0.010534,0.932440,-2.611639,-0.687376,0.006940,0.626980,2.021929


### Example: Filling Missing Values with Group-specific Values

In [48]:
s = Series(randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.056242
2         NaN
3    0.422017
4         NaN
5   -0.709290
dtype: float64

In [49]:
s.fillna(s.mean())

0   -0.447838
1   -1.056242
2   -0.447838
3    0.422017
4   -0.447838
5   -0.709290
dtype: float64

In [50]:
chars = ['a','b','c','d','e','f','g','h']
group_key = ['red']*4 + ['blue']*4

data = Series(randn(8), index=chars)
data[['b','d','g']] = np.nan
data

a   -1.632224
b         NaN
c    1.382446
d         NaN
e   -0.102042
f   -2.497548
g         NaN
h    0.486552
dtype: float64

In [51]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

a   -1.632224
b   -0.124889
c    1.382446
d   -0.124889
e   -0.102042
f   -2.497548
g   -0.704346
h    0.486552
dtype: float64

In [52]:
fill_values = {'red': -0.5, 'blue': 0.5}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

a   -1.632224
b   -0.500000
c    1.382446
d   -0.500000
e   -0.102042
f   -2.497548
g    0.500000
h    0.486552
dtype: float64

### Example: Random Sampling and Permutation

In [53]:
suits = ['H', 'S', 'C', 'D']
card_val = [1,2,3,4,5,6,7,8,9,10,10,10,10]*4
base_names = ['A'] + range(2,11) + ['J','Q','K']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [54]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

draw(deck)

8D     8
JD    10
JS    10
AH     1
6D     6
dtype: int64

In [55]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  3C    3
   7C    7
D  4D    4
   8D    8
H  4H    4
   9H    9
S  4S    4
   7S    7
dtype: int64

### Example: Group Weighted Average and Correlation

In [56]:
df = DataFrame({
    'category': ['a','a','a','a','b','b','b','b'],
    'data': [1]*8,
    'weights': np.random.rand(8)
})

df

,category,data,weights
0,a,1,0.241223
1,a,1,0.782778
2,a,1,0.226840
3,a,1,0.399376
4,b,1,0.091863
5,b,1,0.194029
6,b,1,0.685624
7,b,1,0.864862


In [57]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    1.0
b    1.0
dtype: float64

In [58]:
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px[:4]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93


In [59]:
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [60]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-wise Linear Regression

In [61]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

In [62]:
tips.pivot_table(index=['sex', 'smoker'])

size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [63]:
tips.groupby(['sex','smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [64]:
tips.pivot_table(['tip_pct', 'size'],
                 index=['sex', 'day'],
                 columns='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

In [65]:
tips.groupby(['sex', 'day','smoker'])['tip_pct','size'].mean().unstack()

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

In [66]:
tips.pivot_table(['tip_pct', 'size'],
                 index=['sex','day'],
                 columns='smoker',
                 margins=True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   0.165296  0.209129  0.199388  2.500000  2.000000  2.111111
       Sat   0.147993  0.163817  0.156470  2.307692  2.200000  2.250000
       Sun   0.165710  0.237075  0.181569  3.071429  2.500000  2.944444
       Thur  0.155971  0.163073  0.157525  2.480000  2.428571  2.468750
Male   Fri   0.138005  0.144730  0.143385  2.000000  2.125000  2.100000
       Sat   0.162132  0.139067  0.151577  2.656250  2.629630  2.644068
       Sun   0.158291  0.173964  0.162344  2.883721  2.600000  2.810345
       Thur  0.165706  0.164417  0.165276  2.500000  2.300000  2.433333
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [67]:
tips.pivot_table('tip_pct', 
                 index=['sex','smoker'],
                 columns='day',
                 aggfunc=len,
                 margins=True)

day             Fri   Sat   Sun  Thur    All
sex    smoker                               
Female No       2.0  13.0  14.0  25.0   54.0
       Yes      7.0  15.0   4.0   7.0   33.0
Male   No       2.0  32.0  43.0  20.0   97.0
       Yes      8.0  27.0  15.0  10.0   60.0
All            19.0  87.0  76.0  62.0  244.0

In [68]:
tips.pivot_table('size',
                index=['time','sex','smoker'],
                columns='day',
                aggfunc='sum',
                fill_value=0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

### Cross-Tabulations: Crosstab

In [69]:
data = DataFrame({
    'Sample': [1,2,3,4,5,6,7,8,9,10],
    'Gender': ['F','M','F','M','M','M','F','F','M','F'],
    'Handedness': ['R','L','R','R','L','R','R','L','R','R']
})

data

,Gender,Handedness,Sample
0,F,R,1
1,M,L,2
2,F,R,3
3,M,R,4
4,M,L,5
5,M,R,6
6,F,R,7
7,F,L,8
8,M,R,9
9,F,R,10


In [70]:
pd.crosstab(data.Gender,
            data.Handedness,
            margins=True)

Handedness,L,R,All
Gender,,,
F,1,4,5
M,2,3,5
All,3,7,10


In [71]:
pd.crosstab([tips.time, tips.day],
           tips.smoker,
           margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244